In [1]:
import pandas as pd

In [3]:
dataset_abs_path = '/Users/myZZUNG/myworkspace/posco_ai_challenge/dataset/1. 도전과제/'
dataset_name = '[과제3] 식당 메뉴 및 식사인원.xlsx'

In [2]:
xl = pd.ExcelFile('./[과제3] 식당 메뉴 및 식사인원.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: './[과제3] 식당 메뉴 및 식사인원.xlsx'

In [ ]:
xl.sheet_names

In [ ]:
rest_num = xl.parse('식사인원')
rest_num.info()
rest_num.head()

In [ ]:
rest_num[rest_num[rest_num.columns[0]] == 20030828]

In [ ]:
rest_num.columns

In [ ]:
rest_num[rest_num.columns[1]].value_counts()

### 데이터 설명
* 아침 = 아침식사 + 아침식사1
* 저녁 = 저녁식사 + 저녁식사1
* 점심 = 점심 + 점심1
* 점심(양식) = 점심2

In [ ]:
def parse_foodtime(x):
    if '아침' in x:
        return '아침'
    if '저녁' in x:
        return '저녁'
    if '점심식사2'in x:
        return '점심2'
    if '점심식사' in x:
        return '점심'

In [ ]:
rest_num['time'] = rest_num[rest_num.columns[1]].apply(lambda x: parse_foodtime(x))

In [ ]:
proc_rest_num = rest_num.groupby([rest_num.columns[0],'time']).sum().reset_index()
proc_rest_num.info()

In [ ]:
menu = xl.parse('메뉴')
menu.info()
menu.head()

In [ ]:
menu['식사명'].value_counts()

In [ ]:
menu['time'] = menu['식사명'].apply(lambda x: parse_foodtime(x))
menu['식사내용'] = menu['식사내용'].apply(lambda x: x+',')

In [ ]:
proc_menu = menu.groupby(['일자','time']).sum().drop(['식사명'],axis=1).reset_index()
proc_menu.info()

In [ ]:
proc_menu

In [ ]:
merged_df = proc_menu.merge(proc_rest_num, left_on=['일자','time'], right_on=[proc_rest_num.columns[0],'time'],how='outer')
merged_df.info()

In [ ]:
merged_df[merged_df[merged_df.columns[3]].isnull()]

In [ ]:
train_df = merged_df.dropna().copy()

In [ ]:
train_df.to_csv('train_data.tsv',sep='\t',index=False)